In [5]:
import os
import librosa
import soundfile as sf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from google.colab import drive

# ==========================================
# 1. MOUNT DRIVE VÀ CẤU HÌNH ĐƯỜNG DẪN
# ==========================================
print("🔗 Đang kết nối với Google Drive...")
drive.mount('/content/drive')

BASE_DIR = "/content/drive/MyDrive/Speech_Emotion_Recognition"
INPUT_DIR = os.path.join(BASE_DIR, "input")
RAVDESS_DIR = os.path.join(INPUT_DIR, "ravdess-emotional-speech-audio")
TESS_DIR = os.path.join(INPUT_DIR, "TESS Toronto emotional speech set data")

OUTPUT_DIR = os.path.join(BASE_DIR, "working/processed")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ==========================================
# 2. CẤU HÌNH MAPPING (GỘP NEUTRAL & CALM)
# ==========================================
RAVDESS_MAP = {
    '01': 'neutral', '02': 'neutral', # Gộp Calm vào Neutral
    '03': 'happy',   '04': 'sad',
    '05': 'angry',   '06': 'fearful',
    '07': 'disgust', '08': 'surprised'
}

TESS_MAP = {
    'neutral': 'neutral',
    'happy': 'happy',
    'sad': 'sad',
    'angry': 'angry',
    'fear': 'fearful',
    'disgust': 'disgust',
    'ps': 'surprised'
}

EMOTION_TO_ID = {
    'neutral': 0, 'happy': 1, 'sad': 2, 'angry': 3,
    'fearful': 4, 'disgust': 5, 'surprised': 6
}

# =========================
# 3. HÀM HỖ TRỢ
# =========================
def preprocess_audio(file_path, target_sr=16000):
    try:
        y, sr = librosa.load(file_path, sr=None)
        if y.ndim > 1: y = librosa.to_mono(y)
        y_resampled = librosa.resample(y, orig_sr=sr, target_sr=target_sr)
        return y_resampled, target_sr
    except Exception as e:
        return None, None

# ==========================================
# 4. DUYỆT FILE VÀ TẠO DANH SÁCH GỐC
# ==========================================
all_data = []

print("📂 Đang quét tệp tin dữ liệu...")

# Quét RAVDESS
for root, _, files in os.walk(RAVDESS_DIR):
    for f in files:
        if f.endswith(".wav"):
            parts = f.split('-')
            if len(parts) >= 3:
                emotion_name = RAVDESS_MAP.get(parts[2])
                if emotion_name:
                    all_data.append({
                        'raw_path': os.path.join(root, f),
                        'label_id': EMOTION_TO_ID[emotion_name]
                    })

# Quét TESS
for root, _, files in os.walk(TESS_DIR):
    for f in files:
        if f.endswith(".wav"):
            emo = f.lower().split('.')[0].split('_')[-1]
            emotion_name = TESS_MAP.get(emo)
            if emotion_name:
                all_data.append({
                    'raw_path': os.path.join(root, f),
                    'label_id': EMOTION_TO_ID[emotion_name]
                })

df_raw = pd.DataFrame(all_data)
print(f"✅ Tìm thấy tổng cộng: {len(df_raw)} mẫu.")
print("📊 Phân bổ dữ liệu trước khi cân bằng:\n", df_raw['label_id'].value_counts())

# ==========================================
# 5. CHIA TẬP DỮ LIỆU
# ==========================================
# Chia Train_Full (80%) và Test (20%)
df_train_full, df_test = train_test_split(
    df_raw, test_size=0.2, stratify=df_raw['label_id'], random_state=42
)

# Chia tiếp Validation (15% của Train_Full)
df_train, df_val = train_test_split(
    df_train_full, test_size=0.15, stratify=df_train_full['label_id'], random_state=42
)

# ==========================================
# 6. XỬ LÝ IMBALANCE (OVERSAMPLING TRÊN TRAIN)
# ==========================================
print("\n⚖️ Đang xử lý cân bằng dữ liệu trên tập Train...")
max_size = df_train['label_id'].value_counts().max()
lst_balanced = []

for class_id in df_train['label_id'].unique():
    df_class = df_train[df_train['label_id'] == class_id]
    df_class_oversampled = resample(df_class,
                                    replace=True,      # Nhân bản mẫu
                                    n_samples=max_size, # Đưa về bằng số lượng mẫu lớn nhất
                                    random_state=42)
    lst_balanced.append(df_class_oversampled)

df_train_balanced = pd.concat(lst_balanced)
print("📊 Phân bổ dữ liệu sau khi cân bằng tập Train:\n", df_train_balanced['label_id'].value_counts())

# ==========================================
# 7. TIỀN XỬ LÝ ÂM THANH VÀ LƯU FILE
# ==========================================
def process_and_save_dataset(df, folder_name):
    print(f"🚀 Đang xử lý audio cho tập {folder_name}...")
    # Chỉ lấy các đường dẫn duy nhất để xử lý (tránh xử lý lặp lại file oversampled)
    unique_files = df['raw_path'].unique()
    path_map = {} # Map từ raw_path sang processed_path

    save_path = os.path.join(OUTPUT_DIR, folder_name)
    os.makedirs(save_path, exist_ok=True)

    for i, raw_f in enumerate(unique_files):
        y, sr = preprocess_audio(raw_f)
        if y is not None:
            new_name = f"{folder_name}_{i}.wav"
            full_save_path = os.path.join(save_path, new_name)
            sf.write(full_save_path, y, sr)
            path_map[raw_f] = os.path.join(folder_name, new_name)

        if (i+1) % 400 == 0: print(f"   --> Đã lưu {i+1} file...")

    # Cập nhật đường dẫn mới vào dataframe
    df['file_path'] = df['raw_path'].map(path_map)
    return df.dropna(subset=['file_path'])

# Xử lý và lưu
df_train_final = process_and_save_dataset(df_train_balanced, "train")
df_val_final = process_and_save_dataset(df_val, "val")
df_test_final = process_and_save_dataset(df_test, "test")

# ==========================================
# 8. LƯU CSV CUỐI CÙNG
# ==========================================
cols = ['file_path', 'label_id']
df_train_final[cols].to_csv(os.path.join(OUTPUT_DIR, "train_final.csv"), index=False)
df_val_final[cols].to_csv(os.path.join(OUTPUT_DIR, "val_final.csv"), index=False)
df_test_final[cols].to_csv(os.path.join(OUTPUT_DIR, "test_final.csv"), index=False)

print("\n" + "="*40)
print("🎯 HOÀN TẤT!")
print(f"📊 Kết quả cuối cùng (Số dòng trong CSV):")
print(f"   - Train (Balanced): {len(df_train_final)}")
print(f"   - Val: {len(df_val_final)}")
print(f"   - Test: {len(df_test_final)}")
print("="*40)

🔗 Đang kết nối với Google Drive...
Mounted at /content/drive
📂 Đang quét tệp tin dữ liệu...
✅ Tìm thấy tổng cộng: 4240 mẫu.
📊 Phân bổ dữ liệu trước khi cân bằng:
 label_id
0    688
1    592
3    592
2    592
4    592
6    592
5    592
Name: count, dtype: int64

⚖️ Đang xử lý cân bằng dữ liệu trên tập Train...
📊 Phân bổ dữ liệu sau khi cân bằng tập Train:
 label_id
3    467
5    467
2    467
1    467
0    467
6    467
4    467
Name: count, dtype: int64
🚀 Đang xử lý audio cho tập train...
   --> Đã lưu 400 file...
   --> Đã lưu 800 file...
   --> Đã lưu 1200 file...
   --> Đã lưu 1600 file...
🚀 Đang xử lý audio cho tập val...
   --> Đã lưu 400 file...
🚀 Đang xử lý audio cho tập test...
   --> Đã lưu 400 file...
   --> Đã lưu 800 file...

🎯 HOÀN TẤT!
📊 Kết quả cuối cùng (Số dòng trong CSV):
   - Train (Balanced): 3269
   - Val: 509
   - Test: 848
